# Hugging Face CIFAR-100 Embeddings Example

<div style="display: inline-flex; align-items: center; gap: 10px;">
        <a href="https://colab.research.google.com/github/3lc-ai/3lc-examples/blob/main/example-notebooks/huggingface-cifar100.ipynb"
        target="_blank"
            style="background-color: transparent; text-decoration: none; display: inline-flex; align-items: center;
            padding: 5px 10px; font-family: Arial, sans-serif;"> <img
            src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" style="height: 30px;
            vertical-align: middle;box-shadow: none;"/>
        </a> <a href="https://github.com/3lc-ai/3lc-examples/blob/main/example-notebooks/huggingface-cifar100.ipynb"
            style="text-decoration: none; display: inline-flex; align-items: center; background-color: #ffffff; border:
            1px solid #d1d5da; border-radius: 8px; padding: 2px 10px; color: #333; font-family: Arial, sans-serif;">
            <svg aria-hidden="true" focusable="false" role="img" class="octicon octicon-mark-github" viewBox="0 0 16 16"
            width="20" height="20" fill="#333"
            style="display:inline-block;user-select:none;vertical-align:text-bottom;overflow:visible; margin-right:
            8px;">
                <path d="M8 0c4.42 0 8 3.58 8 8a8.013 8.013 0 0 1-5.45 7.59c-.4.08-.55-.17-.55-.38 0-.27.01-1.13.01-2.2
                0-.75-.25-1.23-.54-1.48 1.78-.2 3.65-.88 3.65-3.95 0-.88-.31-1.59-.82-2.15.08-.2.36-1.02-.08-2.12 0
                0-.67-.22-2.2.82-.64-.18-1.32-.27-2-.27-.68 0-1.36.09-2 .27-1.53-1.03-2.2-.82-2.2-.82-.44 1.1-.16
                1.92-.08 2.12-.51.56-.82 1.28-.82 2.15 0 3.06 1.86 3.75 3.64 3.95-.23.2-.44.55-.51
                1.07-.46.21-1.61.55-2.33-.66-.15-.24-.6-.83-1.23-.82-.67.01-.27.38.01.53.34.19.73.9.82 1.13.16.45.68
                1.31 2.69.94 0 .67.01 1.3.01 1.49 0 .21-.15.45-.55.38A7.995 7.995 0 0 1 0 8c0-4.42 3.58-8 8-8Z"></path>
            </svg> <span style="vertical-align: middle; color: #333;">Open in GitHub</span>
        </a>
</div>

In this notebook we will see how to use a pre-trained Vision Transformers (ViT) model to collect embeddings on the CIFAR-100 dataset.

This notebook demonstrates:

- Registering the `CIFAR-100` dataset from Hugging Face.
- Computing image embeddings with `transformers` and reducing them to 2D with UMAP.
- Adding the computed embeddings as metrics to a 3LC `Run`.

## Project Setup

In [ ]:
PROJECT_NAME = "CIFAR-100"
RUN_NAME = "Collect Image Embeddings"
DESCRIPTION = "Collect image embeddings from ViT model on CIFAR-100"
DEVICE = None
TRAIN_DATASET_NAME = "hf-cifar-100-train"
TEST_DATASET_NAME = "hf-cifar-100-test"
MODEL = "google/vit-base-patch16-224"
BATCH_SIZE = 32
TRANSIENT_DATA_PATH = "../transient_data"
TLC_PUBLIC_EXAMPLES_DEVELOPER_MODE = True
INSTALL_DEPENDENCIES = False

In [ ]:
%%capture
if INSTALL_DEPENDENCIES:
    %pip --quiet install torch --index-url https://download.pytorch.org/whl/cu118
    %pip --quiet install torchvision --index-url https://download.pytorch.org/whl/cu118
    %pip --quiet install 3lc[umap,huggingface]

## Imports

In [ ]:
import logging

import datasets

import tlc

logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)  # Reduce model loading logs
datasets.utils.logging.disable_progress_bar()

## Prepare the data

To read the data into 3LC, we use `tlc.Table.from_hugging_face()` available under the Hugging Face integration. This returns a `Table` that works similarly to a Hugging Face `datasets.Dataset`.

In [ ]:
cifar100_train = tlc.Table.from_hugging_face(
    "cifar100",
    split="train",
    table_name="train",
    project_name=PROJECT_NAME,
    dataset_name=TRAIN_DATASET_NAME,
    description="CIFAR-100 training dataset",
    if_exists="overwrite",
)

cifar100_test = tlc.Table.from_hugging_face(
    "cifar100",
    split="test",
    table_name="test",
    project_name=PROJECT_NAME,
    dataset_name=TEST_DATASET_NAME,
    description="CIFAR-100 test dataset",
    if_exists="overwrite",
)

In [ ]:
cifar100_train[0]["img"]

## Compute the data

We then use the `transformers` library to compute embeddings and `umap-learn` to reduce the embeddings to two dimensions. 

In [ ]:
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from transformers import ViTImageProcessor, ViTModel

if DEVICE is None:
    if torch.cuda.is_available():
        device = "cuda:0"
    elif torch.backends.mps.is_available():
        device = "mps"
    else:
        device = "cpu"
else:
    device = DEVICE

device = torch.device(device)
print(f"Using device: {device}")

In [ ]:
feature_extractor = ViTImageProcessor.from_pretrained(MODEL)
model = ViTModel.from_pretrained(MODEL).to(device)

In [ ]:
def extract_feature(sample):
    return feature_extractor(images=sample["img"], return_tensors="pt")

In [ ]:
def infer_on_dataset(dataset):
    activations = []
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)
    for inputs in tqdm(dataloader, total=len(dataloader)):
        inputs["pixel_values"] = inputs["pixel_values"].squeeze()
        inputs = inputs.to(DEVICE)
        outputs = model(**inputs)
        activations.append(outputs.last_hidden_state[:, 0, :].detach().cpu())

    return activations

In [ ]:
activations = []
model.eval()

for dataset in (cifar100_train, cifar100_test):
    dataset = dataset.map(extract_feature)
    activations.extend(infer_on_dataset(dataset))

In [ ]:
activations = torch.cat(activations).numpy()
activations.shape

In [ ]:
import umap

reducer = umap.UMAP(n_components=2)
embeddings_2d = reducer.fit_transform(activations)

## Collect the embeddings as 3LC metrics

In this example the metrics are contained in a `numpy.ndarray` object. We can specify the schema of this data and provide it directly to 3LC using `Run.add_metrics()`.

In [ ]:
run = tlc.init(
    project_name=PROJECT_NAME,
    run_name=RUN_NAME,
    description=DESCRIPTION,
    if_exists="overwrite",
)

In [ ]:
embeddings_2d_train = embeddings_2d[: len(cifar100_train)]
embeddings_2d_test = embeddings_2d[len(cifar100_train) :]

In [ ]:
embeddings_2d_train = embeddings_2d[: len(cifar100_train)]
embeddings_2d_test = embeddings_2d[len(cifar100_train) :]

In [ ]:
for dataset, embeddings in ((cifar100_train, embeddings_2d_train), (cifar100_test, embeddings_2d_test)):
    run.add_metrics(
        {"embeddings": list(embeddings)},
        column_schemas={"embeddings": tlc.FloatVector2Schema()},
        foreign_table_url=dataset.url,
    )

In [ ]:
run.set_status_completed()